In [ ]:
#Step 1. 구글 코랩에 한글 폰트 설정하기

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
#Step 2.분석할 데이터가 저장된 파일을 불러와서 변수에 할당합니다.
from google.colab import files
myfile = files.upload()
import io
import pandas as pd
#pd.read_csv로 csv파일 불러오기
fish = pd.read_csv(io.BytesIO(myfile['fish.csv']),
                       encoding='cp949')
fish

In [1]:
import pandas as pd

fish = pd.read_csv('fish.csv')
fish

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672


In [2]:
fish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Species   159 non-null    object 
 1   Weight    159 non-null    float64
 2   Length    159 non-null    float64
 3   Diagonal  159 non-null    float64
 4   Height    159 non-null    float64
 5   Width     159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB


In [3]:
fish.describe()

,Weight,Length,Diagonal,Height,Width
count,159.000000,159.000000,159.000000,159.000000,159.000000
mean,398.326415,28.415723,31.227044,8.970994,4.417486
std,357.978317,10.716328,11.610246,4.286208,1.685804
min,0.000000,8.400000,8.800000,1.728400,1.047600
25%,120.000000,21.000000,23.150000,5.944800,3.385650
50%,273.000000,27.300000,29.400000,7.786000,4.248500
75%,650.000000,35.500000,39.650000,12.365900,5.584500
max,1650.000000,63.400000,68.000000,18.957000,8.142000


In [ ]:
# 무게, 길이, 높이 넓이 정보를 가지고, 생선의 종류 예측
data = fish[ ['Weight','Length','Height','Width']].to_numpy()

# 문자열을 숫자로 mapping
target = fish['Species'].map({'Smelt':0, 'Bream':1, 'Roach':2, 'Whitefish':3, 'Parkki':4, 'Perch':5, 'Pike':6})

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=40)

# 무게, 길이, 높이 넓이 정보를 가지고, 생선의 종류 예측
from tensorflow.keras.utils import to_categorical

# 레이블 원-핫 인코딩
train_target = to_categorical(train_target, 7)
test_target = to_categorical(test_target, 7)

# train, test 데이터 shape 확인
print(train_data.shape, train_target.shape)
print(test_data.shape, test_target.shape)

In [ ]:
# 필요한 라이브러리 import
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint

# ModelCheckpoint 콜백 생성
checkpoint = ModelCheckpoint('best_model_fish.keras', save_best_only=True, monitor='var_loss', mode='min')

# 모델 생성
model = Sequential([
    Dense(64, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(32, activation='sigmoid'),
    Dense(16, activation='sigmoid'),
    Dense(7, activation='softmax')
])



# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['accuracy', Precision(), Recall()])

# 모델 학습
model.fit(train_data, train_target, epochs=100, batch_size=64, vaildation_split=0.2)

In [ ]:
from sklearn.metrics import classification_report

# 예측확률을 0, 1값으로 변환
predicted = model.predict(test_data)
predicted_labels = (predicted > 0.5).astype(int)

# classification_report 출력
report = classification_report(test_target, predicted_labels)
print(report)

# Callback 사용

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint

# 모델 생성
model = Sequential([
    Dense(64, activation='relu', input_shape=(train_data.shape[1], )),
    Dense(32, activation='sigmiod'),
    Dense(16, activation='sigmoid'),
    Dense(7, activation='softmax')
])

# 학습된 가중치 불러오기
model.load_weights('best_model_fish.keras')

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['accuracy', Precision(), Recall()])

In [ ]:
# 모델 평가
results = model.evaluate(test_data, test_target)

# 예측
model.predict(tf.constant([[]]))